In [ ]:
import os
import numpy as np
import torch
from PIL import Image
from torchvision import transforms as T
from data_manager import calc_grads
from gradients_to_navigation import Gradients_to_navigation

In [ ]:
dataset_dir   = 'data_grads_to_pixel/'
gradients_dir = "grads/"
pixel_dir     = "pixel/"
image_size    = (64, 64)

In [ ]:
CWD                          = os.getcwd()
INPUT_DATASET_DIR            = os.path.join(CWD,'data/nyuv2/depth')
OUTPUT_DATASET_DIR           = os.path.join(CWD, dataset_dir)
OUTPUT_DATASET_GRADIENTS_DIR = os.path.join(CWD, dataset_dir, gradients_dir)
OUTPUT_DATASET_PIXEL_DIR     = os.path.join(CWD, dataset_dir, pixel_dir)
print(INPUT_DATASET_DIR)
print(OUTPUT_DATASET_DIR)
print(OUTPUT_DATASET_GRADIENTS_DIR)
print(OUTPUT_DATASET_PIXEL_DIR)

In [ ]:
directory = os.fsencode(INPUT_DATASET_DIR)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".png"): 
        depth = Image.open(INPUT_DATASET_DIR + r"/" + filename)
        depth = T.Resize(image_size)(depth)
        depth = T.ToTensor()(depth)
        depth = T.Normalize(mean=(.5,), std=(.5,))(depth)
        x, y = calc_grads(depth)
        torch.save(x, OUTPUT_DATASET_GRADIENTS_DIR + filename[:-4] + '_x.pt')
        torch.save(y, OUTPUT_DATASET_GRADIENTS_DIR + filename[:-4] + '_y.pt')
        x = x.squeeze(0)
        y = y.squeeze(0)
        gradients_to_navigation = Gradients_to_navigation()
        navigation_image = gradients_to_navigation.calculate_navigation_image(x=x, y=y)
        navigate_to_x, navigate_to_y = gradients_to_navigation.calculate_goto_pixel(navigation_image=navigation_image)
        goto_pixel = torch.from_numpy(np.array([navigate_to_x, navigate_to_y]))
        torch.save(goto_pixel, OUTPUT_DATASET_PIXEL_DIR + filename[:-4] + '.pt')
#         x = torch.load(OUTPUT_DATASET_DIR + '_x.pt')
#         print(type(x))
#         print(x.shape)
#         print(x)